In [2]:
import pandas as pd
import numpy as np


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

In [22]:
df=pd.read_csv('tested.csv')

In [23]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [24]:
df.drop(columns=['PassengerId',	'Name','Ticket','Cabin'],inplace=True)

In [41]:
x_train,test,Y_train,Test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [42]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [43]:
Y_train.sample(3)

105    0
332    0
374    1
Name: Survived, dtype: int64

In [44]:
t1=ColumnTransformer([('q',SimpleImputer(),[2]),
                     ('r',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

In [60]:
t2=ColumnTransformer([('s',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])],remainder='passthrough')

In [61]:
t3=ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))])

In [62]:
t4=DecisionTreeClassifier()

In [66]:
pipe=Pipeline([('t1',t1),
               ('t2',t2),
               ('t3',t3),
               ('t4',t4)])

In [67]:
pipe.fit(x_train,Y_train)

Pipeline(steps=[('t1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('q', SimpleImputer(), [2]),
                                                 ('r',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('t2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('s',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('t3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('t4', DecisionTreeClassifier())])

In [68]:
pipe.named_steps['t1'].transformers_[0][1].statistics_

array([30.11576336])

In [69]:
y_predict=pipe.predict(test)

In [70]:
y_predict

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(Test,y_predict)

0.5952380952380952

In [74]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))